## Basic Applications Powered by LLMs with LangChain

### Introduction

LangChain is designed to assist developers in building end-to-end applications using language models. It offers an array of tools, components, and interfaces that simplify the process of creating applications powered by large language models and chat models. LangChain streamlines managing interactions with LLMs, chaining together multiple components, and integrating additional resources, such as APIs and databases. Having gained a foundational understanding of the library in previous lesson, let's now explore various examples of utilizing prompts to accomplish multiple tasks.

A key feature of LangChain is its support for prompts, which encompasses prompt management, prompt optimization, and a generic interface for all LLMs. The framework also provides common utilities for working with LLMs.

ChatPromptTemplate is used to create a structured conversation with the AI model, making it easier to manage the flow and content of the conversation. In LangChain, message prompt templates are used to construct and work with prompts, allowing us to exploit the underlying chat model's potential fully.

System and Human prompts differ in their roles and purposes when interacting with chat models. SystemMessagePromptTemplate provides initial instructions, context, or data for the AI model, while HumanMessagePromptTemplate are messages from the user that the AI model responds to.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())

print(response.content)

Could not import azure.core python package.


"Inception" is a science fiction action film directed by Christopher Nolan. It was released in 2010 and stars Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and Marion Cotillard.

The movie follows Dom Cobb (played by DiCaprio), a skilled thief who specializes in extracting valuable information from the subconscious of his targets by entering their dreams. Cobb is given a chance to redeem himself by performing the opposite task of "inception" – planting an idea in someone's mind instead of stealing it.

The film explores the concept of shared dreaming and delves into the complexities of the human mind. It takes the audience on a thrilling journey through various dream levels, blurring the lines between reality and the dream world.

"Inception" received critical acclaim for its originality, visual effects, and thought-provoking storyline. It was praised for its complex narrative structure and the performances of the cast. The film was a commercial success, grossing over

## Summarization chain example:

LangChain prompts can be found in various use cases, such as summarization or question-answering chains. For example, when creating a summarization chain, LangChain enables interaction with an external data source to fetch data for use in the generation step. This could involve summarizing a lengthy piece of text or answering questions using specific data sources.

The following code will initialize the language model using OpenAI class with a temperature of 0 - because we want deterministic output.  The load_summarize_chain function accepts an instance of the language model and returns a pre-built summarization chain. Lastly, the PyPDFLoader class is responsible for loading PDF files and converting them into a format suitable for processing by LangChain. 

In [ ]:
# Import necessary modules
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

# Initialize language model
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Load the summarization chain
summarize_chain = load_summarize_chain(llm)

# Load the document using PyPDFLoader
document_loader = PyPDFLoader(file_path="data/Retrieve rerank generate.pdf")
document = document_loader.load()

# Summarize the document
summary = summarize_chain(document)
print(summary['output_text'])

Using a web URL as document:

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

#loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader = WebBaseLoader("https://medium.com/towards-artificial-intelligence/gptq-quantization-on-a-llama-2-7b-fine-tuned-model-with-huggingface-a7b291fbb871")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)

'This article discusses the GPTQ quantization method for compressing large language models. GPTQ is a post-training quantization method that can compress models to 2, 3, or 4 bits per parameter without sacrificing accuracy. The article explains the process of quantizing a fine-tuned Llama 2 7B model using the GPTQ methodology integrated with the transformers library. It also highlights the advantages and limitations of GPTQ and provides instructions on how to load and use the quantized model. The author shares their experience with quantizing the model and compares the performance of the quantized model with the original model.'

In [1]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import WebBaseLoader

# Initialize language model
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Load the summarization chain
summarize_chain = load_summarize_chain(llm, chain_type="stuff")

# Load the document using WebBaseLoader
loader = WebBaseLoader("https://medium.com/towards-artificial-intelligence/gptq-quantization-on-a-llama-2-7b-fine-tuned-model-with-huggingface-a7b291fbb871/")
docs = loader.load()

# Summarize the document
summary = summarize_chain(docs)
print(summary['output_text'])

As we mention early, we can reach the limit of the context length. We will deal with this issue in a later lesson

## QA chain example:

We can also use LangChain to manage prompts for asking general questions from the LLMs. These models are proficient in addressing fundamental inquiries. Nevertheless, it is crucial to remain mindful of the potential issue of hallucinations, where the models may generate non-factual information. To address this concern, we will later introduce the Retrieval chain as a means to overcome this problem.

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

llm = OpenAI(model_name="text-davinci-003", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

We define a custom prompt template by creating an instance of the PromptTemplate class. The template string contains a placeholder {question} for the input question, followed by a newline character and the "Answer:" label.  The input_variables argument is set to the list of available placeholders in the prompt (like a question in this case) to indicate the name of the variable that the chain will replace in the template.run() method.

We then instantiate an OpenAI model named text-davinci-003 with a temperature of 0. The OpenAI class is used to create the instance, and the model_name and temperature arguments are provided. Finally, we create a question-answering chain using the LLMChain class. 

The class constructor takes two arguments: llm, which is the instantiated OpenAI model, and prompt, which is the custom prompt template we defined earlier. 

By following these steps, we can process input questions effectively with the custom question-answering, generating appropriate answers using the OpenAI model and the custom prompt template.

In [10]:
chain.run("what is the meaning of life?")

' The meaning of life is subjective and can vary from person to person. For some, it may be to find happiness and fulfillment, while for others it may be to make a difference in the world. Ultimately, the meaning of life is up to each individual to decide.'